# TSTR SDV Dataset E

In [2]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
print('Libraries imported!!')

Libraries imported!!


In [3]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = 'EVALUATION FUNCTIONS/UTILITY'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for data labelling analisys
from utility_evaluation import DataPreProcessor
from utility_evaluation import train_evaluate_model

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read data

In [4]:
#read real dataset
train_data = pd.read_csv(HOME_PATH + 'SYNTHETIC DATASETS/SDV/E_PimaIndiansDiabetes_Synthetic_SDV.csv')
categorical_columns = ['Outcome']
for col in categorical_columns :
    train_data[col] = train_data[col].astype('category')
train_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,3,145,129,37,230,41.9,0.854,34,1
1,8,112,77,38,108,37.7,1.187,38,0
2,1,114,64,44,260,30.7,0.065,24,0
3,3,194,83,29,293,35.5,0.689,24,0
4,2,70,65,26,-132,24.4,0.129,35,0
...,...,...,...,...,...,...,...,...,...
609,10,100,66,37,95,42.4,0.093,34,0
610,0,122,77,-2,180,14.2,0.246,34,0
611,6,126,62,32,119,25.4,0.673,48,0
612,2,181,81,28,118,34.5,0.888,38,1


In [5]:
#read test data
test_data = pd.read_csv(HOME_PATH + 'REAL DATASETS/TEST DATASETS/E_PimaIndiansDiabetes_Real_Test.csv')
for col in categorical_columns :
    test_data[col] = test_data[col].astype('category')
test_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...,...
149,9,165,88,0,0,30.4,0.302,49,1
150,1,77,56,30,56,33.3,1.251,24,0
151,8,95,72,0,0,36.8,0.485,57,0
152,2,146,70,38,360,28.0,0.337,29,1


In [6]:
target = 'Outcome'
#quick look at the breakdown of class values
print('Train data')
print(train_data.shape)
print(train_data.groupby(target).size())
print('#####################################')
print('Test data')
print(test_data.shape)
print(test_data.groupby(target).size())

Train data
(614, 9)
Outcome
0    455
1    159
dtype: int64
#####################################
Test data
(154, 9)
Outcome
0    99
1    55
dtype: int64


## 2. Pre-process training data

In [7]:
target = 'Outcome'
categorical_columns = None
numerical_columns = train_data.select_dtypes(include=['int64','float64']).columns.tolist()
categories = None

data_preprocessor = DataPreProcessor(categorical_columns, numerical_columns, categories)
x_train = data_preprocessor.preprocess_train_data(train_data.loc[:, train_data.columns != target])
y_train = train_data.loc[:, target]

x_train.shape, y_train.shape

((614, 8), (614,))

## 3. Preprocess test data

In [8]:
x_test = data_preprocessor.preprocess_test_data(test_data.loc[:, test_data.columns != target])
y_test = test_data.loc[:, target]
x_test.shape, y_test.shape

((154, 8), (154,))

## 4. Create a dataset to save the results

In [9]:
results = pd.DataFrame(columns = ['model','accuracy','precision','recall','f1'])
results

,model,accuracy,precision,recall,f1


## 4. Train and evaluate Random Forest Classifier

In [10]:
rf_results = train_evaluate_model('RF', x_train, y_train, x_test, y_test)
results = results.append(rf_results, ignore_index=True)
rf_results

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished


,model,accuracy,precision,recall,f1
0,RF,0.7208,0.723,0.7208,0.6886


## 5. Train and Evaluate KNeighbors Classifier

In [11]:
knn_results = train_evaluate_model('KNN', x_train, y_train, x_test, y_test)
results = results.append(knn_results, ignore_index=True)
knn_results

,model,accuracy,precision,recall,f1
0,KNN,0.6883,0.6932,0.6883,0.6312


## 6. Train and evaluate Decision Tree Classifier

In [12]:
dt_results = train_evaluate_model('DT', x_train, y_train, x_test, y_test)
results = results.append(dt_results, ignore_index=True)
dt_results

,model,accuracy,precision,recall,f1
0,DT,0.7143,0.707,0.7143,0.7086


## 7. Train and evaluate Support Vector Machines Classifier

In [13]:
svm_results = train_evaluate_model('SVM', x_train, y_train, x_test, y_test)
results = results.append(svm_results, ignore_index=True)
svm_results

[LibSVM]

,model,accuracy,precision,recall,f1
0,SVM,0.4545,0.516,0.4545,0.4624


## 8. Train and evaluate Multilayer Perceptron Classifier

In [14]:
mlp_results = train_evaluate_model('MLP', x_train, y_train, x_test, y_test)
results = results.append(mlp_results, ignore_index=True)
mlp_results

Iteration 1, loss = 0.68600972
Iteration 2, loss = 0.63112641
Iteration 3, loss = 0.59592846
Iteration 4, loss = 0.57059009
Iteration 5, loss = 0.55093981
Iteration 6, loss = 0.53546930
Iteration 7, loss = 0.52324380
Iteration 8, loss = 0.51424131
Iteration 9, loss = 0.50761188
Iteration 10, loss = 0.50355189
Iteration 11, loss = 0.49968641
Iteration 12, loss = 0.49748692
Iteration 13, loss = 0.49443833
Iteration 14, loss = 0.49036675
Iteration 15, loss = 0.48777434
Iteration 16, loss = 0.48430242
Iteration 17, loss = 0.48337700
Iteration 18, loss = 0.48471694
Iteration 19, loss = 0.48349634
Iteration 20, loss = 0.48025099
Iteration 21, loss = 0.47656698
Iteration 22, loss = 0.47431296
Iteration 23, loss = 0.47124892
Iteration 24, loss = 0.47010242
Iteration 25, loss = 0.46756322
Iteration 26, loss = 0.46577180
Iteration 27, loss = 0.46428279
Iteration 28, loss = 0.46236898
Iteration 29, loss = 0.46099066
Iteration 30, loss = 0.45726967
Iteration 31, loss = 0.45701651
Iteration 32, los

,model,accuracy,precision,recall,f1
0,MLP,0.6494,0.6367,0.6494,0.6402


## 9. Save results file

In [15]:
results.to_csv('RESULTS/models_results_sdv.csv', index=False)
results

,model,accuracy,precision,recall,f1
0,RF,0.7208,0.7230,0.7208,0.6886
1,KNN,0.6883,0.6932,0.6883,0.6312
2,DT,0.7143,0.7070,0.7143,0.7086
3,SVM,0.4545,0.5160,0.4545,0.4624
4,MLP,0.6494,0.6367,0.6494,0.6402
